# Data Optimization - Johnson & Johnson

## Leitura dos arquivos

In [1]:
import pandas as pd
import glob

files = glob.glob('./data/cvtJnJVisionFull03-SEP-2020050447_01.txt');

df = pd.read_csv(files[0], sep='|');

df.head(5)

C:\Users\ccper\AppData\Local\Temp\ipykernel_23112\549375973.py:6: DtypeWarning: Columns (2,4,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(files[0], sep='|');


,poNumber,poName,poRequisitionerWwid,poRequisitionerName,poPreparerWwid,poPreparerName,costCenter,costCenterDesc,primaryInternalOrder,profitCenter,...,shipTo,deliverTo,commodityType,excludeDownpaymentRequestsForPayments,sourceSystemApprovableId,matOrSrc,requisitionNumber,receivableIndicator,poLineNumber,splitLineNumber
0,991533176,SAFETY GLASSES & SHOES - SOLID DOSE PACKAGING ...,169596,W BRADLEY,21606,PATRICIA DELP,63616,SOLID DOSE PKG,000900003174,0000156000,...,7050 Camp Hill Road Fort Washington PA,0.0,INDIRECT,N,NaN,S,PR1496395,N,1,1
1,552573383,Cotización para estudio de ruido,78003909,JORGE ZAMUDIO,78003909,JORGE ZAMUDIO,13820250,S&E,NaN,NaN,...,Circuito Interior Norte No 1820 Parque Juarez CH,0.0,INDIRECT,N,NaN,S,PR10042258-V2,Y,1,1
2,993988771,18-1543 [ Order 3Pi # 34590 ] 86517 - Break Fi...,196395,THOMAS MCGEOUGH,702087536,DORA VILLARREAL,#,Not Available,70007514,141002,...,A Johnson Johnson Company15715 Arrow Irwinda...,6.0,INDIRECT,N,PPPI,M,PR9861888,Y,2,1
3,552639550,Dr Acevedo - Programa de Residentes en Artrosc...,1011106,YAMILA DER,1011106,YAMILA DER,1000006339,MedicalAffairsDePuy,NaN,NaN,...,AV. ITALIA 7519 OFICINA 301 Montevideo MO,0.0,INDIRECT,N,OnDemand,S,PR10243168,N,1,1
4,P23331262R,Codes 667113+996655+996656,10012070,BRUNO STARAJ,10012070,BRUNO STARAJ,51621,Pack & Label BE,NaN,NaN,...,Kempenlaan 1 Beerse #,0.0,DIRECT,N,OnDemand,M,PR4835255,N,4,1


## Agrupando dados

Requisitos:

Group rows by

- poNumber
- costCenter
- primaryInternalOrder
- profitCenter
- generalLedgerAccount
- needByDate
- poEndDate
- poStartDate
- receivableIndicator
- projectWbs
- matOrSrc
- accountingActivityCode

In [2]:
accrual = df.groupby(['poNumber', 'costCenter', 'primaryInternalOrder',
'profitCenter', 'generalLedgerAccount', 'needByDate',
'poEndDate', 'poStartDate', 'receivableIndicator',
'projectWbs', 'matOrSrc'], dropna=False, as_index=False)

In [3]:
accrual.count()

,poNumber,costCenter,primaryInternalOrder,profitCenter,generalLedgerAccount,needByDate,poEndDate,poStartDate,receivableIndicator,projectWbs,...,region,businessArea,shipTo,deliverTo,commodityType,excludeDownpaymentRequestsForPayments,sourceSystemApprovableId,requisitionNumber,poLineNumber,splitLineNumber
0,551807444,#,NaN,CONSUMER,5129000114,2014-01-15,NaN,NaN,Y,NaN,...,1,1,1,1,1,1,0,1,1,1
1,551966878,#,NaN,0021110163,6051390108,2015-01-26,NaN,NaN,Y,NaN,...,1,1,1,1,1,1,0,1,1,1
2,551992702,1000001211,NaN,NaN,0620120001,2015-03-30,2015-12-31,2015-03-26,N,NaN,...,12,12,12,12,12,12,0,12,12,12
3,551992703,1000001211,NaN,NaN,0620120001,2015-03-30,2015-12-31,2015-03-26,N,NaN,...,12,12,12,12,12,12,0,12,12,12
4,551993689,1000001533,006000000023,NaN,0620010002,2015-04-01,2015-09-26,2015-09-22,N,NaN,...,4,4,4,4,4,4,0,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77422,P23335960R,8768,NaN,NaN,51100300,2020-12-31,2020-12-31,NaN,D,NaN,...,1,1,1,1,1,1,1,1,1,1
77423,P23335965R,NaN,NaN,NaN,60600031,2020-09-07,2020-09-07,NaN,D,NaN,...,1,1,1,1,1,1,1,1,1,1
77424,P23335973N,2030002302,NaN,NaN,65010003,2020-09-30,2020-09-30,NaN,N,NaN,...,1,1,1,1,1,1,1,1,1,1
77425,P23335981N,51600,NaN,NaN,63560004,2020-09-09,NaN,NaN,N,NaN,...,1,1,1,1,1,1,1,1,1,1


In [4]:
df.columns

Index(['poNumber', 'poName', 'poRequisitionerWwid', 'poRequisitionerName',
       'poPreparerWwid', 'poPreparerName', 'costCenter', 'costCenterDesc',
       'primaryInternalOrder', 'profitCenter', 'generalLedgerAccount',
       'generalLedgerAccountDesc', 'projectWbs', 'supplierNumber',
       'supplierName', 'supplierEmailAddress', 'poType', 'poStatus',
       'poCloseStatus', 'poCreationDate', 'poStartDate', 'poEndDate',
       'receiptDates', 'invoiceDates', 'invoicePaidStatus', 'transactionDate',
       'clearingDocumentRef', 'clearingDateReference', 'needByDate',
       'localCurrencyForPoValue', 'documentCurrencyForPoValue',
       'localCurrencyForGoodsReceipt', 'documentCurrencyForGoodsReceipt',
       'localCurrencyForInvoiceReceipt', 'docCurrencyForInvoiceReceipt',
       'poValueInGlobalCurrency', 'poValueInLocalCurrency',
       'poValueInDocCurrency', 'gdsReceiptValueInGlobalCurrency',
       'gdsReceiptValueInlocalCurrency', 'gdsReceiptValueInDocCurrency',
       'invoice

No momento de agrupar, é necessário somar os campos que serão mesclados.

`SUM(poValueInGlobalCurrency)`

`SUM(poValueInLocalCurrency)`

`SUM(poValueInDocCurrency)`

`SUM(gdsReceiptValueInGlobalCurrency)`

`SUM(gdsReceiptValueInlocalCurrency)`

`SUM(gdsReceiptValueInDocCurrency)`

`SUM(invoiceReceiptValueInGlobalCurrency)`

`SUM(invoiceReceiptValueInLocalCurrency)`

`SUM(invoiceReceiptValueInDocCurrency)`

`SUM(deliverTo)`

Fazer persistir colunas que somem após operação de soma, fazer como:
`'columnName': 'first'` na propriedade do aggreggate `.agg()`

In [5]:
new_df = accrual.agg({
    'poName': 'first',
    'poRequisitionerWwid': 'first',
    'poRequisitionerWwid': 'first',
    'poRequisitionerName': 'first',
    'poPreparerWwid': 'first',
    'poPreparerName': 'first',
    'costCenterDesc': 'first',
    'generalLedgerAccountDesc': 'first',
    'projectWbs': 'first',
    'supplierNumber': 'first',
    'supplierName': 'first',
    'supplierEmailAddress': 'first',
    'poType': 'first',
    'poStatus': 'first',
    'poCloseStatus': 'first',
    'poCreationDate': 'first',
    'receiptDates': 'first',
    'invoiceDates': 'first',
    'invoicePaidStatus': 'first',
    'transactionDate': 'first',
    'clearingDocumentRef': 'first',
    'clearingDateReference': 'first',
    'localCurrencyForPoValue': 'first',
    'documentCurrencyForPoValue': 'first',
    'localCurrencyForGoodsReceipt': 'first',
    'documentCurrencyForGoodsReceipt': 'first',
    'localCurrencyForInvoiceReceipt': 'first',
    'docCurrencyForInvoiceReceipt': 'first',
    'poValueInGlobalCurrency': 'first',
    'poValueInLocalCurrency': 'first',
    'poValueInDocCurrency': 'first',
    'gdsReceiptValueInGlobalCurrency': 'first',
    'gdsReceiptValueInlocalCurrency': 'first',
    'gdsReceiptValueInDocCurrency': 'first',
    'invoiceReceiptValueInGlobalCurrency': 'first',
    'invoiceReceiptValueInLocalCurrency': 'first',
    'invoiceReceiptValueInDocCurrency': 'first',
    'aribaBu': 'first',
    'mrc': 'first',
    'companyCode': 'first',
    'legalEntity': 'first',
    'fsid': 'first',
    'region': 'first',
    'businessArea': 'first',
    'shipTo': 'first',
    'deliverTo': 'first',
    'commodityType': 'first',
    'excludeDownpaymentRequestsForPayments': 'first',
    'sourceSystemApprovableId': 'first',
    'requisitionNumber': 'first',
    'receivableIndicator': 'first',
    'poLineNumber': 'first',
    'splitLineNumber': 'first',
    
    'poValueInGlobalCurrency': 'sum',
    'poValueInLocalCurrency': 'sum',
    'poValueInDocCurrency': 'sum',
    'gdsReceiptValueInGlobalCurrency': 'sum',
    'gdsReceiptValueInlocalCurrency': 'sum',
    'gdsReceiptValueInDocCurrency': 'sum',
    'invoiceReceiptValueInGlobalCurrency': 'sum',
    'invoiceReceiptValueInLocalCurrency': 'sum',
    'invoiceReceiptValueInDocCurrency': 'sum',
    'deliverTo': 'sum'
    })

In [6]:
new_df.head(5)

,poNumber,costCenter,primaryInternalOrder,profitCenter,generalLedgerAccount,needByDate,poEndDate,poStartDate,matOrSrc,poName,...,businessArea,shipTo,deliverTo,commodityType,excludeDownpaymentRequestsForPayments,sourceSystemApprovableId,requisitionNumber,receivableIndicator,poLineNumber,splitLineNumber
0,551807444,#,NaN,CONSUMER,5129000114,2014-01-15,NaN,NaN,M,COMPRAS VIRTUALES EXITO (ADICIONAL),...,#1,Calle 15 31146NIT 890.101.815 9 Acopi Yumbo VA,5718654.0,INDIRECT,N,None,PR7231147-V2,Y,1,1
1,551966878,#,NaN,0021110163,6051390108,2015-01-26,NaN,NaN,M,Acondicionamiento Código 58117 CAREFREE TANGA ...,...,#1,Calle 15 31146NIT 890.101.815 9 Acopi Yumbo VA,106260.0,BOTH,N,None,PR7843337,Y,1,1
2,551992702,1000001211,NaN,NaN,0620120001,2015-03-30,2015-12-31,2015-03-26,S,PMM PLUS,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,BOTH,N,None,PR7947924,N,8,1
3,551992703,1000001211,NaN,NaN,0620120001,2015-03-30,2015-12-31,2015-03-26,S,NON RETAIL BAE DE DATOS,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,BOTH,N,None,PR7947833,N,10,1
4,551993689,1000001533,006000000023,NaN,0620010002,2015-04-01,2015-09-26,2015-09-22,S,HEMTPOCME 082 - International Myeloma Workshop,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,INDIRECT,N,None,PR7943812-V2,N,1,1


## Salvando arquivos

In [7]:
new_df.to_csv('./data/accruedDataJnJ.csv', index=False, sep='|')

In [8]:
modified_df = pd.read_csv('./data/accruedDataJnJ.csv', sep='|')

modified_df.head(5)

C:\Users\ccper\AppData\Local\Temp\ipykernel_23112\2678250046.py:1: DtypeWarning: Columns (0,4,7,10,12,15,16,19,20,26,45,46,47,48,51) have mixed types. Specify dtype option on import or set low_memory=False.
  modified_df = pd.read_csv('./data/accruedDataJnJ.csv', sep='|')


,poNumber,costCenter,primaryInternalOrder,profitCenter,generalLedgerAccount,needByDate,poEndDate,poStartDate,matOrSrc,poName,...,businessArea,shipTo,deliverTo,commodityType,excludeDownpaymentRequestsForPayments,sourceSystemApprovableId,requisitionNumber,receivableIndicator,poLineNumber,splitLineNumber
0,551807444,#,NaN,CONSUMER,5129000114,2014-01-15,NaN,NaN,M,COMPRAS VIRTUALES EXITO (ADICIONAL),...,#1,Calle 15 31146NIT 890.101.815 9 Acopi Yumbo VA,5718654.0,INDIRECT,N,NaN,PR7231147-V2,Y,1,1
1,551966878,#,NaN,0021110163,6051390108,2015-01-26,NaN,NaN,M,Acondicionamiento Código 58117 CAREFREE TANGA ...,...,#1,Calle 15 31146NIT 890.101.815 9 Acopi Yumbo VA,106260.0,BOTH,N,NaN,PR7843337,Y,1,1
2,551992702,1000001211,NaN,NaN,620120001,2015-03-30,2015-12-31,2015-03-26,S,PMM PLUS,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,BOTH,N,NaN,PR7947924,N,8,1
3,551992703,1000001211,NaN,NaN,620120001,2015-03-30,2015-12-31,2015-03-26,S,NON RETAIL BAE DE DATOS,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,BOTH,N,NaN,PR7947833,N,10,1
4,551993689,1000001533,006000000023,NaN,620010002,2015-04-01,2015-09-26,2015-09-22,S,HEMTPOCME 082 - International Myeloma Workshop,...,#1,"Boulevard Adolfo Ruiz Cortines 3720, To Delega...",0.0,INDIRECT,N,NaN,PR7943812-V2,N,1,1


## Gerando amostras de dados

Aqui serão geradas amostras de dados para fins de estudo. Seleciona-se as 10 POs que mais se repetiram e as 10 que menos se repetiram.

In [48]:
accruedFrames = [
    new_df.sort_values(['poValueInGlobalCurrency'], ascending=False).head(10), 
    new_df.sort_values(['poValueInGlobalCurrency'], ascending=True).head(10)
]

rawFrames = []

for accruedFrame in accruedFrames:
    for index, row in accruedFrame.iterrows():
        isNull = row.isnull();
        rawFrames.append(df.loc[
            ((df['poNumber'] == row['poNumber']) | (isNull['poNumber'] & df['poNumber'].isnull())) &
            ((df['costCenter'] == row['costCenter'])  | (isNull['costCenter'] & df['costCenter'].isnull())) &
            ((df['primaryInternalOrder'] == row['primaryInternalOrder'])  | (isNull['primaryInternalOrder'] & df['primaryInternalOrder'].isnull())) &
            ((df['profitCenter'] == row['profitCenter'])  | (isNull['profitCenter'] & df['profitCenter'].isnull())) &
            ((df['generalLedgerAccount'] == row['generalLedgerAccount'])  | (isNull['generalLedgerAccount'] & df['generalLedgerAccount'].isnull())) &
            ((df['needByDate'] == row['needByDate'])  | (isNull['needByDate'] & df['needByDate'].isnull())) &
            ((df['poEndDate'] == row['poEndDate'])  | (isNull['poEndDate'] & df['poEndDate'].isnull())) &
            ((df['receivableIndicator'] == row['receivableIndicator'])  | (isNull['receivableIndicator'] & df['receivableIndicator'].isnull())) &
            ((df['projectWbs'] == row['projectWbs'])  | (isNull['projectWbs'] & df['projectWbs'].isnull())) &
            ((df['matOrSrc'] == row['matOrSrc'])  | (isNull['matOrSrc'] & df['matOrSrc'].isnull()))
        ])

pd.concat(rawFrames).to_csv('./data/sampleData-rawPOs.csv', sep='|', index=False)
pd.concat(accruedFrames).to_csv('./data/sampleData-accruedPOs.csv', sep='|', index=False)


In [ ]:
new_df.sort_values(['poValueInGlobalCurrency'], ascending=True).head(10)